In [3]:
from collections import OrderedDict
import pandas as pd
import pickle

from pee_stego_v4 import PEEStego

VERSION = 4

# Set up patient

In [4]:
from utils.data_preparation import get_filenames_from_folder, slice_batch_data

# Tentukan folder path tempat secret
patient_folder_path = 'data/mit-bih-arrhythmia-database-1.0.0/'

# Dapatkan semua nama file .dat dari folder
patient_codes = get_filenames_from_folder('dat', patient_folder_path)
patient_codes.sort()

signals = []
for patient_code in patient_codes:
    batch_signals = slice_batch_data(patient_code=patient_code, max_batch=1)
    signals.extend(batch_signals)

print(len(signals))

46


# Run model test

In [5]:
from utils.telegram_bot import send_message
from datetime import datetime

stego = PEEStego()
right_now = datetime.now()
test_result = []

secret_name = "secret"
secret_data = "1" * 100_000

try:
    for payload_rate in [1, 2, 3]:
        for threshold in [0, 1]:
            for index_signal, original_signal in enumerate(signals):
                print(f"Pada payload_rate {payload_rate} threshold {threshold} signals {index_signal} secret {secret_name}")
                print(">>Embedding...")
                watermarked_signal, mirror_data, last_phase, last_i, last_embedded_bit_total, result = stego.embed(
                    original_signal, secret_data, payload_rate=payload_rate, threshold=threshold)
                print(f"Last Phase: {last_phase}, Last I: {last_i}, Last embedded bit: {last_embedded_bit_total}")
                print(">>Extracting...")
                extracted_original_signal, extracted_secret_data = stego.extract(
                    watermarked_signal, mirror_data, last_phase, last_i, last_embedded_bit_total, payload_rate=payload_rate, threshold=threshold)

                print(">>", original_signal[0:20], secret_data[0:20])
                print(">>", extracted_original_signal[0:20],
                    extracted_secret_data[0:20])
                print(">>", "Len -> Ori:", len(original_signal), ", Ex-ori:", len(extracted_original_signal),
                    ", Sec:", len(
                            secret_data), ", Ex-sec:", len(extracted_secret_data))
                print("******\n******")

                test_result.append([payload_rate, threshold, index_signal, secret_name, len(secret_data), len(extracted_secret_data), result.ncc, result.prd, result.snr, result.timer])
except Exception as err:
    print(repr(err))
    send_message(f"Terjadi masalah {right_now}")


Pada payload_rate 1 threshold 0 signals 0 secret secret
>>Embedding...
[-145 -145 -145 -145 -145 -145 -146 -146 -119 -134]
Total yang kurang dari sama dengan threshold (T = 1): 480
Unhidden secret: 0
NCC: 0.9999854816932195
PRD: 0.25672721477603827
SNR: 51.8105618162416
PSNR: 30.140759135519257
Time: 0.17201733589172363
Last Phase: 3, Last I: 3593, Last embedded bit: 1
>>Extracting...
[-145 -145 -145 -145 -145 -145 -146 -145 -119 -134]
[-145 -145 -145 -145 -145 -145 -145 -145 -119 -135]
[-145 -145 -145 -145 -145 -145 -145 -145 -120 -135]
>> [-145 -145 -145 -145 -145 -145 -145 -145 -120 -135 -145 -150 -160 -155
 -160 -175 -180 -185 -170 -155] 11111111111111111111
>> [-145 -145 -145 -145 -145 -145 -145 -145 -120 -135 -145 -150 -160 -155
 -160 -175 -180 -185 -170 -155] 11111111111111111111
>> Len -> Ori: 3600 , Ex-ori: 3600 , Sec: 100000 , Ex-sec: 3116
******
******
Pada payload_rate 1 threshold 0 signals 1 secret secret
>>Embedding...
[-345 -345 -345 -345 -345 -345 -346 -346 -329 -319]
T

In [6]:
out_csv = f"result_v{VERSION}_2_{right_now.strftime('%Y%m%d_%H%M%S.csv')}"

In [7]:
df = pd.DataFrame(test_result, columns=['payload_rate', 'threshold', 'index_signal', 'secret_name', 'len_secret_data', 'len_extracted_secret_data', 'ncc', 'prd', 'snr', 'time'])
df.to_csv(f'out/{out_csv}', index=False)